# 04 - Time Series Analysis, Modeling, and Forecasting (Bilingual)

Este notebook é bilíngue (PT/EN).
This notebook is bilingual (PT/EN).

Objetivo/Goal: exemplos completos de análise, modelagem e previsão de séries temporais com Prophet e ARIMA, decomposição, visualizações, tratamento de datas, métricas especializadas e exportação de forecasts.


In [ ]:
# !pip install pandas numpy matplotlib seaborn plotly statsmodels prophet --quiet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings; warnings.filterwarnings('ignore')

# Prophet import (try/catch for environments without it)
try:
    from prophet import Prophet
except Exception as e:
    Prophet = None
    print('Prophet not available, continuing without Prophet:', e)


## 1) Data loading and date handling / Carregamento e tratamento de datas
- Parse datetime, set frequency, handle missing values.
- Fazer parse de datas, definir frequência e tratar faltantes.


In [ ]:
# Example synthetic daily time series / Série diária sintética
rng = pd.date_range('2018-01-01', '2022-12-31', freq='D')
np.random.seed(42)
trend = np.linspace(10, 50, len(rng))
seasonal = 10 * np.sin(2 * np.pi * rng.dayofyear / 365.25)
noise = np.random.normal(0, 2, len(rng))
y = trend + seasonal + noise
df = pd.DataFrame({'ds': rng, 'y': y})

# Introduce missing dates and values / Introduzir datas e valores faltantes
df_missing = df.copy().sample(frac=0.98, random_state=1).sort_values('ds').reset_index(drop=True)
# Reindex to daily frequency and fill missing via interpolation
df_full = (df_missing.set_index('ds')
           .asfreq('D')
           .interpolate('time')
           .reset_index())
df_full.rename(columns={'index': 'ds'}, inplace=True)
df_full.head()


## 2) Visualization / Visualização
- Static and interactive plots.
- Gráficos estáticos e interativos.


In [ ]:
fig, ax = plt.subplots(figsize=(12,4))
ax.plot(df_full['ds'], df_full['y'], label='y')
ax.set_title('Time Series (Daily) / Série Temporal (Diária)')
ax.legend(); plt.show()

px.line(df_full, x='ds', y='y', title='Interactive Time Series').show()


## 3) Decomposition / Decomposição
- Trend, seasonality, residuals.
- Tendência, sazonalidade e resíduos.


In [ ]:
result = seasonal_decompose(df_full.set_index('ds')['y'], model='additive', period=365)
result.plot().set_size_inches(12,8)
plt.suptitle('Additive Decomposition / Decomposição Aditiva', y=1.02)
plt.show()


## 4) Stationarity check / Verificação de estacionariedade (ADF)


In [ ]:
adf_stat, pvalue, _, _, crit, _ = adfuller(df_full['y'])
print(f'ADF: {adf_stat:.3f} | p-value: {pvalue:.3f}')
print('Critical values:', crit)
print('Stationary' if pvalue < 0.05 else 'Non-stationary')


## 5) ARIMA modeling / Modelagem ARIMA
- Simple ARIMA as example (p,d,q).
- ARIMA simples como exemplo (p,d,q).


In [ ]:
train = df_full.iloc[:-365].copy()
test = df_full.iloc[-365:].copy()

model = ARIMA(train.set_index('ds')['y'], order=(5,1,2))
arima_res = model.fit()
print(arima_res.summary())

# Forecast horizon equal to test length
forecast = arima_res.forecast(steps=len(test))
pred = pd.DataFrame({'ds': test['ds'].values, 'yhat': forecast.values})

fig, ax = plt.subplots(figsize=(12,4))
ax.plot(train['ds'], train['y'], label='train')
ax.plot(test['ds'], test['y'], label='test')
ax.plot(pred['ds'], pred['yhat'], label='ARIMA forecast')
ax.legend(); ax.set_title('ARIMA Forecast vs Test')
plt.show()


## 6) Prophet modeling (if available) / Modelagem com Prophet (se disponível)
- Daily seasonality + yearly seasonality.
- Sazonalidade diária e anual.


In [ ]:
if Prophet is not None:
    m = Prophet(daily_seasonality=True, yearly_seasonality=True, weekly_seasonality=True)
    m.fit(train[['ds','y']])
    future = m.make_future_dataframe(periods=len(test), freq='D')
    fcst = m.predict(future)
    fcst_tail = fcst.tail(len(test))[['ds','yhat','yhat_lower','yhat_upper']]
    
    fig1 = m.plot(fcst)
    plt.title('Prophet Forecast')
    plt.show()
    fig2 = m.plot_components(fcst)
    plt.show()
else:
    print('Prophet not installed; skipping Prophet section.')


## 7) Specialized metrics / Métricas especializadas
- MAE, RMSE, MAPE, sMAPE.


In [ ]:
def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / np.clip(np.abs(y_true), 1e-8, None))) * 100

def smape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + 1e-8))

arima_metrics = {
    'MAE': mean_absolute_error(test['y'], pred['yhat']),
    'RMSE': np.sqrt(mean_squared_error(test['y'], pred['yhat'])),
    'MAPE': mape(test['y'], pred['yhat']),
    'sMAPE': smape(test['y'], pred['yhat'])
}
print('ARIMA metrics / Métricas ARIMA:', arima_metrics)

if Prophet is not None:
    merged = test.merge(fcst_tail[['ds','yhat']], on='ds', how='left')
    prop_metrics = {
        'MAE': mean_absolute_error(merged['y'], merged['yhat']),
        'RMSE': np.sqrt(mean_squared_error(merged['y'], merged['yhat'])),
        'MAPE': mape(merged['y'], merged['yhat']),
        'sMAPE': smape(merged['y'], merged['yhat'])
    }
    print('Prophet metrics / Métricas Prophet:', prop_metrics)


## 8) Export forecasts / Exportação de previsões
- Save CSV with forecast and intervals.
- Salvar CSV com previsões e intervalos.


In [ ]:
export_arima = pred.rename(columns={'yhat': 'yhat_arima'})
export_arima.to_csv('forecast_arima.csv', index=False)
print('Saved forecast_arima.csv')

if Prophet is not None:
    export_prophet = fcst_tail.rename(columns={'yhat':'yhat_prophet'})
    export_prophet.to_csv('forecast_prophet.csv', index=False)
    print('Saved forecast_prophet.csv')


## 9) Next steps / Próximos passos
- Hyperparameter tuning (p,d,q), seasonal ARIMA (SARIMA), cross-validation.
- Ajuste de hiperparâmetros, SARIMA e validação cruzada.
